In [8]:
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt
import yfinance as yf


In [12]:
#fetch data

api = tradeapi.REST('PK0LC2J1NSMU3IUAXF31', 'VpfhPftx0Wvmkn5PoOKabFcUUHrCSxQ8LF8dri3s', base_url='https://paper-api.alpaca.markets')

def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data


In [13]:
#strategy

def compute_indicators(data):
    data['SMA_50'] = data['close'].rolling(window=50).mean()
    data['SMA_200'] = data['close'].rolling(window=200).mean()
    data['momentum'] = data['close'].pct_change(periods=20)
    data['mean_reversion'] = data['close'] - data['SMA_50']
    return data

def generate_signals(data):
    data['signal'] = 0
    data['signal'][data['SMA_50'] > data['SMA_200']] = 1  # Buy signal
    data['signal'][data['SMA_50'] < data['SMA_200']] = -1  # Sell signal
    return data


In [14]:
#backtest

def backtest(data):
    data['returns'] = data['Close'].pct_change()
    data['strategy_returns'] = data['returns'] * data['signal'].shift(1)
    cumulative_returns = (1 + data['strategy_returns']).cumprod()
    return cumulative_returns

symbol = 'AAPL'
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

data = fetch_data(symbol, start_date, end_date)
data = compute_indicators(data)
data = generate_signals(data)
cumulative_returns = backtest(data)
print(cumulative_returns.tail())

[*********************100%%**********************]  1 of 1 completed


KeyError: 'close'

In [ ]:
#results

plt.figure(figsize=(10, 5))
cumulative_returns.plot()
plt.title(f'Cumulative Returns for {symbol}')
plt.show()
